# Import các thư viện

In [ ]:
import matplotlib.pyplot as plt
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from ucimlrepo import fetch_ucirepo
from graphviz import Source
from IPython.display import display, Image
import seaborn as sns
import category_encoders as ce

# Heart Desease Dataset

## 1. Chuẩn bị dataset

### 1.1. Train và test dataset

In [ ]:
# Fetch Heart Disease dataset
heart_disease = fetch_ucirepo(id=45)

# Extract features and labels
features = heart_disease.data.features.values
labels = np.ravel(heart_disease.data.targets)

# Convert labels to binary: 0 (no disease), 1 (disease)
labels_binary = np.where(labels > 0, 1, 0)

# Handle missing values
data = pd.concat([pd.DataFrame(features, columns=heart_disease.data.features.columns), pd.Series(labels_binary, name='num')], axis=1)
data = data.dropna().reset_index(drop=True)
features = data.iloc[:, :-1].values
labels_binary = data.iloc[:, -1].values

# Encode the target variable (though not strictly necessary for binary 0/1)
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels_binary)

# Define class names for binary classification
class_names = ['No Disease', 'Disease']

# Define train/test split proportions
split_ratios = [(0.4, 0.6), (0.6, 0.4), (0.8, 0.2), (0.9, 0.1)]

subsets = []

for split_ratio in split_ratios:
    feature_train, feature_test, label_train, label_test = train_test_split(
        features, labels_encoded, test_size=split_ratio[1], random_state=42, stratify=labels_encoded
    )
    
    subsets.append({
        'feature_train': feature_train,
        'label_train': label_train,
        'feature_test': feature_test,
        'label_test': label_test
    })

### 1.2. Visualization

In [ ]:
# Visualize the class distribution in the original dataset
plt.figure(figsize=(10, 5))
bins = np.arange(len(np.unique(labels_encoded)) + 1) - 0.5
plt.hist(labels_encoded, bins=bins, color="green", alpha=0.7, edgecolor="black")
plt.title("Class Distribution in the Original Dataset")
plt.xlabel("Classes")
plt.ylabel("Frequency")
plt.xticks(np.arange(len(class_names)), class_names)
plt.show()

# Visualize distributions for each train/test split
for i in range(len(subsets)):
    label_train = subsets[i]['label_train']
    label_test = subsets[i]['label_test']

    bins = np.arange(len(np.unique(labels_encoded)) + 1) - 0.5
    
    plt.figure(figsize=(10, 5))
    plt.hist(label_train, bins=bins, color="blue", alpha=0.7, edgecolor="black", label="Training")
    plt.title(f"Class Distribution for {split_ratios[i][0]}/{split_ratios[i][1]} Split (Training)")
    plt.xlabel("Classes")
    plt.ylabel("Frequency")
    plt.xticks(np.arange(len(class_names)), class_names)
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(10, 5))
    plt.hist(label_test, bins=bins, color="orange", alpha=0.7, edgecolor="black", label="Testing")
    plt.title(f"Class Distribution for {split_ratios[i][0]}/{split_ratios[i][1]} Split (Testing)")
    plt.xlabel("Classes")
    plt.ylabel("Frequency")
    plt.xticks(np.arange(len(class_names)), class_names)
    plt.legend()
    plt.show()

## 2. Xây dựng Decision Tree

### 2.1. Train model

In [ ]:
# Train and evaluate the Decision Tree model using Entropy (Information Gain)
models = []
for i, subset in enumerate(subsets):
    feature_train = subset['feature_train']
    label_train = subset['label_train']
    # model = DecisionTreeClassifier(criterion='entropy', random_state=42, max_depth=5, class_weight='balanced')
    model = DecisionTreeClassifier(criterion='entropy', random_state=42)
    model.fit(feature_train, label_train)
    models.append(model)

### 2.2. Visualization

In [ ]:
for i in range(len(models)):
    print(f"Decision tree of the model trained with split ratio {split_ratios[i][0]}/{split_ratios[i][1]}")
    # Export the decision tree to DOT format
    dot_data = export_graphviz(
        models[i],  # Use the correct model from the list
        out_file=None,  # Don't save to file, we will use the source in memory
        feature_names=heart_disease.data.features.columns.values,  # Use Heart Disease feature names
        class_names=class_names,  # Convert class names to strings
        filled=True,
        rounded=True,
        special_characters=True,
        fontname="Arial"
    )
    
    # Render the DOT file with Graphviz
    graph = Source(dot_data)
    # Visualize the tree
    graph.render(f"./tree/tree1/tree_{split_ratios[i][0]}_{split_ratios[i][1]}", 
                 format='png', 
                 cleanup=True)
    display(Image(f"./tree/tree1/tree_{split_ratios[i][0]}_{split_ratios[i][1]}.png"))

## 3. Đánh giá Decision Tree

### 3.1. Classification report & confusion matrix

In [ ]:
# For each model and split, make predictions, generate a report, and confusion matrix
for i, subset in enumerate(subsets):
    feature_train = subset['feature_train']
    label_train = subset['label_train']
    feature_test = subset['feature_test']
    label_test = subset['label_test']
    
    # Make predictions
    label_pred = models[i].predict(feature_test)
    
    # Print class distribution in test set
    print(f"Split {split_ratios[i]} Class Distribution in Test Set:")
    print(pd.Series(label_test).value_counts())
    
    # Generate classification report
    print(f"Classification Report for {split_ratios[i][0]}/{split_ratios[i][1]} Split:")
    print(classification_report(label_test, label_pred, target_names=class_names, zero_division=0))
    
    # Generate confusion matrix
    cm = confusion_matrix(label_test, label_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    plt.title(f"Confusion Matrix for {split_ratios[i][0]}/{split_ratios[i][1]} Split")
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

### 3.2. Insights

#### 3.2.1. Tỷ lệ 40/60

- Độ chính xác: 78% trên 179 mẫu kiểm tra.
- F1-score trung bình (macro/weighted): khoảng 0.77–0.78.
- Lớp “Không bệnh”: F1-score = 0.80, recall = 0.82, mô hình nhận diện khá tốt.
- Lớp “Có bệnh”: F1-score = 0.75, recall = 0.72, vẫn còn bỏ sót khoảng 28% số bệnh nhân.
- Mô hình thiên nhẹ về lớp “Không bệnh”, nhưng vẫn duy trì hiệu suất tổng thể khá ổn định.

#### 3.2.2. Tỷ lệ 60/40


- Độ chính xác: 75% trên 119 mẫu kiểm tra.
- Lớp “Không bệnh”: recall cao (0.86), cho thấy mô hình phát hiện tốt các ca không bệnh.
- Lớp “Có bệnh”: recall giảm còn 0.62, mô hình bỏ sót nhiều hơn so với tỷ lệ 40/60.
- F1-score cả hai lớp đều giảm nhẹ, hiệu suất không được cải thiện rõ dù tăng dữ liệu huấn luyện.
- Mô hình có xu hướng thiên về lớp “Không bệnh”, cần điều chỉnh để cân bằng hơn.

#### 3.2.3. Tỷ lệ 80/20

- Độ chính xác: 77% trên 60 mẫu kiểm tra.
- Lớp “Không bệnh”: F1-score = 0.77.
- Lớp “Có bệnh”: recall cải thiện rõ rệt lên 0.79 → mô hình phát hiện nhiều bệnh nhân hơn.
- Mô hình duy trì được độ chính xác tốt, F1-score hai lớp cân bằng hơn.
- Đây là tỷ lệ có hiệu suất ổn định và hiệu quả nhất trong các thử nghiệm.

#### 3.2.4. Tỷ lệ 90/10

- Độ chính xác: 70% trên 30 mẫu kiểm tra – thấp nhất trong 4 tỷ lệ.
- Lớp “Không bệnh”: recall = 0.81, precision giảm còn 0.68, dễ nhầm lẫn.
- Lớp “Có bệnh”: recall chỉ đạt 0.57, gần một nửa số bệnh nhân không được phát hiện.
- F1-score dao động từ 0.64–0.74, cho thấy độ ổn định không cao.
- Kích thước test set nhỏ gây nhiễu trong đánh giá, không phản ánh đúng hiệu suất thực tế.

## 4. Độ sâu (depth) và độ chính xác (accuracy) của Decision Tree

### 4.1 Visualization

In [ ]:
# Select the 80/20 split (index 2 in split_ratios)
subset_80_20 = subsets[2]
feature_train_80_20 = subset_80_20['feature_train']
label_train_80_20 = subset_80_20['label_train']
feature_test_80_20 = subset_80_20['feature_test']
label_test_80_20 = subset_80_20['label_test']

accuracy_scores = []
depths = [None, 2, 3, 4, 5, 6, 7]

for depth in depths:
    model = DecisionTreeClassifier(criterion='entropy', random_state=42, max_depth=depth, class_weight='balanced')
    model.fit(feature_train_80_20, label_train_80_20)

    print(f"Decision tree of the model trained with split ratio 80/20 and max depth {depth}")
    # Export the decision tree to DOT format
    dot_data = export_graphviz(
        model,
        out_file=None,
        feature_names=heart_disease.data.features.columns,
        class_names=class_names,
        filled=True,
        rounded=True,
        special_characters=True,
        fontname="Arial"
    )
    
    # Render the DOT file with Graphviz
    graph = Source(dot_data)
    # Visualize the tree
    graph.render(f"./tree_80_20/tree1/tree_{depth}", 
                 format='png', 
                 cleanup=True)
    display(Image(filename=f"./tree_80_20/tree1/tree_{depth}.png"))

    # Make predictions and calculate accuracy
    pred = model.predict(feature_test_80_20)
    accuracy = accuracy_score(label_test_80_20, pred)
    print(f"Accuracy for max_depth {depth}: {accuracy:.4f}")
    accuracy_scores.append(accuracy)

# Plot the results
depths_for_plot = [str(d) if d is not None else 'None' for d in depths]
plt.figure(figsize=(10, 5))
plt.plot(depths_for_plot, accuracy_scores, marker='o', linestyle='-', color='blue')
plt.title('Accuracy vs Max Depth for Decision Tree (80/20 Split)')
plt.xlabel('Max Depth')
plt.ylabel('Accuracy')
plt.xticks(depths_for_plot)
plt.grid(True)
plt.show()

### 4.2. Insights

| max_depth | None  | 2     | 3     | 4     | 5     | 6     | 7     |
|-----------|-------|-------|-------|-------|-------|-------|-------|
| Accuracy  | 0.774 | 0.730 | 0.797 | 0.775 | 0.778 | 0.771 | 0.773 |

- Độ chính xác cao nhất (~79.67%) đạt được khi giới hạn **max_depth = 3**.
- Khi **max_depth = None**, độ chính xác giảm nhẹ (~77.4%), có dấu hiệu overfitting nhẹ.
- Với **max_depth = 2**, mô hình underfitting rõ rệt (độ chính xác chỉ ~73%).
- Từ độ sâu 4–7, hiệu suất dao động nhẹ (77–78%), không cải thiện so với độ sâu 3.
- **Kết luận:** Độ sâu = 3 là tối ưu nhất, cân bằng giữa khả năng học và tránh overfitting.